# SQL Server Troubleshooting Guide 

## Tampa Bay SQL Server User Group (TSUG)

Last Updated 02/21/2023

### Troubleshooting Process

*Source - https://www.ciscopress.com/articles/article.asp?p=2999386&seqNum=5*

General troubleshooting process steps are as follows:

Step 1. Identify the symptoms of the problem.

Step 2. Establish a theory of probable cause.

Step 3. Investigate the theory to determine the cause.

Step 4. Establish a plan of action to resolve the problem and implement the solution.

Step 5. Verify full system functionality and, if applicable, implement preventive measures.

Step 6. Document findings, actions, and outcomes.

### SQL Server Instance Information and Status

In [ ]:
/* Find running services on the server 
Source: https://learn.microsoft.com/en-us/sql/relational-databases/system-dynamic-management-views/system-dynamic-management-views?view=sql-server-ver16
*/

SELECT *
FROM sys.dm_server_services;
GO

### SQL Server Databases Information and Status

In [ ]:
/* Databases and their statuses
Source: https://learn.microsoft.com/en-us/sql/relational-databases/system-compatibility-views/sys-sysdatabases-transact-sql?view=sql-server-ver16 
*/

DECLARE @status INT 
SELECT @status = status FROM sys.sysdatabases WHERE name = DB_NAME() 
PRINT DB_NAME() + ' - ' + CONVERT(VARCHAR(20),@status) 

IF ( (1 & @status) = 1 ) PRINT 'autoclose' 
IF ( (2 & @status) = 2 ) PRINT '2' 
IF ( (4 & @status) = 4 ) PRINT 'select into/bulkcopy' 
IF ( (8 & @status) = 8 ) PRINT 'trunc. log on chkpt' 
IF ( (16 & @status) = 16 ) PRINT 'torn page detection' 
IF ( (32 & @status) = 32 ) PRINT 'loading' 
IF ( (64 & @status) = 64 ) PRINT 'pre recovery' 
IF ( (128 & @status) = 128 ) PRINT 'recovering' 
IF ( (256 & @status) = 256 ) PRINT 'not recovered' 
IF ( (512 & @status) = 512 ) PRINT 'offline' 
IF ( (1024 & @status) = 1024 ) PRINT 'read only' 
IF ( (2048 & @status) = 2048 ) PRINT 'dbo use only' 
IF ( (4096 & @status) = 4096 ) PRINT 'single user' 
IF ( (8192 & @status) = 8192 ) PRINT '8192 not sure' 
IF ( (16384 & @status) = 16384 ) PRINT '16384 not sure' 
IF ( (32768 & @status) = 32768 ) PRINT 'emergency mode' 
IF ( (65536 & @status) = 65536 ) PRINT 'online' 
IF ( (131072 & @status) = 131072 ) PRINT '131072' 
IF ( (262144 & @status) = 262144 ) PRINT '2621442' 
IF ( (524288 & @status) = 524288 ) PRINT '524288' 
IF ( (1048576 & @status) = 1048576 ) PRINT '1048576' 
IF ( (2097152 & @status) = 2097152 ) PRINT '2097152' 
IF ( (4194304 & @status) = 4194304 ) PRINT 'autoshrink' 
IF ( (1073741824 & @status) = 1073741824 ) PRINT 'cleanly shutdown' 

### Database Backups Status

In [ ]:
/* Check the backups before you do anything else!
Source: https://dba.stackexchange.com/questions/150241/t-sql-query-for-date-of-last-full-backup-size-and-location
*/
WITH LastBackUp AS
(
SELECT  bs.database_name,
        bs.backup_size,
        bs.backup_start_date,
        bmf.physical_device_name,
        Position = ROW_NUMBER() OVER( PARTITION BY bs.database_name ORDER BY bs.backup_start_date DESC )
FROM  msdb.dbo.backupmediafamily bmf
JOIN msdb.dbo.backupmediaset bms ON bmf.media_set_id = bms.media_set_id
JOIN msdb.dbo.backupset bs ON bms.media_set_id = bs.media_set_id
WHERE   bs.[type] = 'D'
AND bs.is_copy_only = 0
)
SELECT 
        sd.name AS [Database],
        CAST(backup_size / 1048576 AS DECIMAL(10, 2) ) AS [BackupSizeMB],
        backup_start_date AS [Last Full DB Backup Date],
        physical_device_name AS [Backup File Location]
FROM sys.databases AS sd
LEFT JOIN LastBackUp AS lb
    ON sd.name = lb.database_name
    AND Position = 1
ORDER BY [Database];

### SQL Server Logs

In [ ]:
/* Read Error Logs in SQL Server
Source: https://learn.microsoft.com/en-us/troubleshoot/sql/database-engine/startup-shutdown/manage-error-log/ 
0 means current log, use higher numbers to go backwards
*/
EXEC sp_readerrorlog 0, 1;
GO

### SQL Server Agent Logs

In [ ]:
/* Read Error Logs in SQL Server
Source: https://learn.microsoft.com/en-us/troubleshoot/sql/database-engine/startup-shutdown/manage-error-log/ 
0 means current log, use higher numbers to go backwards
*/
EXEC sp_readerrorlog 0, 2;
GO